In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create dataframe from boxing data csv 'bouts_out_new.csv'

df = pd.read_csv('bouts_out_new.csv')

# Create a dictionary with each numerical column name, and the acceptable range of values for that column.

num_cols_dict = {col: (df[col].quantile(0.01), df[col].quantile(0.99)) for col in df.describe().columns}

# Use the col_ranges dictionary to replace all unacceptable values with NaN, 
# and then fill the NaN values with the median of each column.

for col in num_cols_dict.keys():
    df[col] = df[col].where((df[col] >= num_cols_dict[col][0]) & (df[col] <= num_cols_dict[col][1]))
    df[col] = df[col].fillna(df[col].median())
    
# Remove the "stance" column, as there are too many null values. 
# Remove the "judge" columns, since the "result" column already denotes the winner.

df = df.drop(['stance_A','stance_B', 'judge1_A', 'judge1_B', 'judge2_A', 'judge2_B','judge3_A','judge3_B'], axis=1)

